# AI 기술 발전의 이해


---
---



## **1. 1950s~1980s: 규칙 기반** 전문가 시스템

- **시나리오**: 체온, 기침/근육통/콧물 여부로 **감기 의심**을 규칙으로 판정
- **목표**: 규칙 설계의 장단점 체감(명확하지만 유연성/확장성 한계)

In [ ]:
# 규칙 기반 진단기 (전문가 시스템 미니 버전)
# Colab/Jupyter에서 바로 실행 가능

from dataclasses import dataclass

@dataclass
class Patient:
    name: str
    temperature: float  # 체온(℃)
    cough: bool         # 기침
    muscle_pain: bool   # 근육통
    runny_nose: bool    # 콧물

def diagnose_cold(p: Patient) -> str:
    """
    간단 규칙:
    - 고열(>= 37.8) + (기침 or 근육통 or 콧물) -> '감기 의심'
    - 미열(37.3~37.7) + 증상 2개 이상 -> '감기 의심'
    - 그 외 -> '정상 또는 기타'
    """
    symptoms = sum([p.cough, p.muscle_pain, p.runny_nose])
    if p.temperature >= 37.8 and symptoms >= 1:
        return "감기 의심"
    if 37.3 <= p.temperature < 37.8 and symptoms >= 2:
        return "감기 의심"
    return "정상 또는 기타"

# 테스트
patients = [
    Patient("Alice", 38.1, True, False, False),
    Patient("Bob", 37.5, True, True, False),
    Patient("Charlie", 36.8, False, False, True),
    Patient("Dana", 37.2, True, True, True),
]

for p in patients:
    print(f"{p.name}: {diagnose_cold(p)}")

# ▶ 확장실험:
# 1) 규칙을 더 세밀화해보세요(예: 나이/기저질환).
# 2) 모호 케이스(37.4℃, 증상 1개)를 어떻게 처리할지 팀 토론.


- **핵심 포인트**
    -“설계자가 지식(규칙)을 직접 코딩” → 설명가능성↑, 데이터 불필요
    - 그러나 새 증상/예외가 늘면 규칙 폭증 → 유지보수/일반화 한계



---



## **2. 1990s: 고전 ML(SVM)으로 2D 분류 + 결정경계 시각화**

- **시나리오**: 두 개의 2D 가우시안 분포에서 샘플을 뽑아 **SVM**으로 분류
- **목표**: 데이터 → 학습 → 검증 → 결정경계 시각화의 표준 워크플로 이해

- **SVM(Support Vector Machine)**
    - 분류(classification) 및 회귀(regression) 분석을 위한 강력한 지도 학습(supervised learning) 모델
    - 분류 문제에서 뛰어난 성능을 보이며
    - 데이터 포인트들을 효과적으로 분리하는 최적의 경계(결정 경계, decision boundary)를 찾아내는 것이 핵심 원리

In [ ]:
#!pip install scikit-learn matplotlib numpy

In [ ]:
# SVM 분류 (1990s 고전 ML 스타일)

import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# 1) 합성 데이터 생성 (두 개의 가우시안 분포)
np.random.seed(42)
n = 400
mean1, cov1 = [0, 0], [[1.0, 0.3], [0.3, 1.0]]
mean2, cov2 = [2.5, 2.0], [[1.0, -0.2], [-0.2, 1.2]]
X1 = np.random.multivariate_normal(mean1, cov1, n//2)
X2 = np.random.multivariate_normal(mean2, cov2, n//2)
X = np.vstack([X1, X2])
y = np.array([0]*(n//2) + [1]*(n//2))


# 2) 학습/검증 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# 3) SVM 모델 학습 (RBF 커널)
model = SVC(kernel='rbf', gamma='scale', C=1.0)
model.fit(X_train, y_train)


# 4) 검증
pred = model.predict(X_test)
acc = accuracy_score(y_test, pred)
print(f"Test Accuracy: {acc:.3f}")


# 5) 결정경계 시각화
def plot_decision_boundary(clf, X, y, title="SVM Decision Boundary"):
    x_min, x_max = X[:,0].min()-1, X[:,0].max()+1
    y_min, y_max = X[:,1].min()-1, X[:,1].max()+1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300),
                         np.linspace(y_min, y_max, 300))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    plt.contourf(xx, yy, Z, alpha=0.25)
    plt.scatter(X[:,0], X[:,1], c=y, s=18, edgecolors='k')
    plt.title(title)
    plt.xlabel("x1"); plt.ylabel("x2")
    plt.show()

plot_decision_boundary(model, X, y, title=f"SVM (RBF) | Test Acc={acc:.3f}")


# ▶ 확장실험:
# - 커널 변경(linear, poly) 및 C, gamma 조절 → 결정경계 비교
# - 표준화(Scaling) 전/후 성능 비교


- **핵심 포인트**
    - 데이터로부터 경계(결정 함수)를 학습
    - 하이퍼파라미터(C, gamma)가 일반화 성능에 큰 영향
    - 시각화로 모델 복잡도 ↔ 과적합 감각 익히기



---



## **3. 2010s 이후: 딥러닝(MLP 분류)**

- **시나리오**: 간단한 “손글씨 비슷한” 8×8 합성 이미지 데이터(숫자 0~9)를 **다층 퍼셉트론(MLP)** 으로 분류
- **목표**: 딥러닝의 기본 학습 루프/과적합 관리(검증셋, EarlyStopping) 체험

In [ ]:
#!pip install tensorflow scikit-learn matplotlib

In [ ]:
# 딥러닝(2010s): Keras MLP로 8x8 손글씨(MNIST) 숫자 분류

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 1) 데이터 로드 (8x8 이미지, 1797 샘플)
digits = load_digits()
X = digits.images  # (n, 8, 8)
y = digits.target  # (n,)


# 2) 전처리: 평탄화 + 정규화
X = X.reshape(len(X), -1).astype("float32")  # (n, 64)
scaler = StandardScaler()
X = scaler.fit_transform(X)


# 3) 학습/검증/테스트 분리
X_train, X_tmp, y_train, y_tmp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_tmp, y_tmp, test_size=0.5, random_state=42, stratify=y_tmp)


# 4) MLP 모델 정의
model = keras.Sequential([
    layers.Input(shape=(64,)),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.2),
    layers.Dense(64, activation="relu"),
    layers.Dropout(0.2),
    layers.Dense(10, activation="softmax"),
])

model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])


# 5) EarlyStopping으로 과적합 제어
early_stop = keras.callbacks.EarlyStopping(
    monitor="val_accuracy", patience=10, restore_best_weights=True
)

# 6) 학습
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=64,
    callbacks=[early_stop],
    verbose=0
)

print(f"Best Val Acc: {max(history.history['val_accuracy']):.3f}")
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Acc: {test_acc:.3f}")


# 7) 학습 곡선 시각화
plt.figure()
plt.plot(history.history["accuracy"], label="train_acc")
plt.plot(history.history["val_accuracy"], label="val_acc")
plt.xlabel("epoch"); plt.ylabel("accuracy"); plt.legend()
plt.title("MLP Training Curve")
plt.show()


# 8) 예측 확인
probs = model.predict(X_test[:8], verbose=0)
preds = probs.argmax(axis=1)
print("Pred:", preds)
print("True:", y_test[:8].tolist())


In [ ]:
# 9) 새로운 데이터 예측 함수
def predict_digit(model, scaler, input_array):
    """
    Parameters
    ----------
    model : 학습된 Keras 모델
    scaler : 학습 시 사용한 스케일러 (fit된 상태)
    input_array : np.ndarray (64,) 형태의 1차원 배열 (8x8 이미지 평탄화된 형태)

    Returns
    -------
    pred_class : int 예측된 숫자 클래스 (0~9)
    pred_probs : np.ndarray 각 클래스별 확률 (길이 10)
    """
    # 입력을 2차원으로 reshape (모델 입력 크기 맞추기)
    # 학습 때와 동일한 정규화 적용
    arr = np.array(input_array).reshape(1, -1).astype("float32")
    arr = scaler.transform(arr)
    # 예측
    probs = model.predict(arr, verbose=0)
    pred_class = probs.argmax(axis=1)[0]
    return pred_class, probs[0]


# 테스트: X_test에서 첫 샘플을 임의로 넣어보기
sample = X_test[0] * 1.0  # 복사
pred_class, pred_probs = predict_digit(model, scaler, sample)
print("예측된 숫자:", pred_class)
print("클래스별 확률:", np.round(pred_probs, 3))
print("실제 정답:", y_test[0])


- **핵심 포인트**
    - 표현 학습: 모델이 특징을 자동 추출 (수작업 특징보다 유연)
    - 일반화: 드롭아웃, EarlyStopping 등으로 과적합 방지
    - 실전 확장: CNN(이미지), RNN/Transformer(시계열/자연어)로 확장 가능



---



## **4. LLM(Large Language Model) 맛보기**

### **transformers** 라이브러리


- **Hugging Face가 개발한 오픈소스 라이브러리(Transformer 모델 라이브러리)**
- 자연어 처리, 컴퓨터 비전, 음성 등 다양한 분야의 최신 Transformer 기반 사전 학습된(pretrained) 모델을 쉽고 빠르게 사용할 수 있도록 지원함
- 3개의 딥러닝 라이브러리 지원 :  PyTorch, TensorFlow, Jax
- 상세설명 : https://pypi.org/project/transformers/



- **지원 플랫폼 / 요구사항**
    - Python 3.9 이상, PyTorch 2.1+, TensorFlow 2.6+, Flax 0.4.1+
- **용도 / 특징**
    - 텍스트, 이미지, 오디오, 멀티모달 모델들처럼 다양한 영역의 pretrained 모델을 간단한 API(pipeline)로 빠르게 사용 가능
- **추천 사용 케이스**
    - 빠르고 쉽게 NLP, 컴퓨터 비전, 음성 등의 사전학습 모델을 사용하거나 fine-tuning 하고자 할 때

- **설치 방법**

In [ ]:
# 설치된 transformers 버전 확인  -->(ex: Version: 4.55.4)
!pip show transformers | grep Version

In [ ]:
# 다른 모델과 호환성 때문에 transformers 4.48.3 버전을 사용.
!pip install transformers==4.48.3

In [ ]:
# 설치된 PyTorch 버전 확인  -->(ex: Version: 2.8.0+cu126 (+cu126-->CUDA 12.6 버전 지원))
!pip show torch | grep Version

# pip install torch==2.8.0

### **QuickTour**

#### **감정분석**

In [ ]:
# 감정 분석 파이프라인
from transformers import pipeline

classifier = pipeline('sentiment-analysis')
print(classifier('We are very happy to introduce pipeline to the transformers repository.'))

In [ ]:
from transformers import pipeline

nlp = pipeline("sentiment-analysis")
print(nlp("이 강의는 정말 재미있어요!"))


#### **이미지 객체 감지**

In [ ]:
import requests
from PIL import Image, ImageDraw
from transformers import pipeline

# 이미지 다운로드
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/coco_sample.png"
image_data = requests.get(url, stream=True).raw
image = Image.open(image_data)

# 객체 감지 파이프라인
object_detector = pipeline("object-detection")
results = object_detector(image)

# 원본 이미지에 바운딩 박스 그리기
draw = ImageDraw.Draw(image)

for result in results:
    box = result["box"]
    label = result["label"]
    score = result["score"]

    # 박스 좌표
    x1, y1, x2, y2 = box["xmin"], box["ymin"], box["xmax"], box["ymax"]

    # 박스 그리기
    draw.rectangle(((x1, y1), (x2, y2)), outline="red", width=3)
    # 레이블 + 점수 표시
    draw.text((x1, y1 - 10), f"{label} {score:.2f}", fill="yellow")

# 결과 출력 (Colab 환경에서 시각화)
image.show()   # 로컬 환경이면 새 창
display(image) # Colab/Jupyter 환경이면 노트북 안에 표시


 #### **[참고] Access Token 넣고 비공개 모델 접근**
- https://huggingface.co/settings/tokens  로그인 후 사용하고자 하는 모델에 대한 access용 토큰 발급

In [ ]:
# 1단계: Hugging Face 토큰 생성 및 설정
from huggingface_hub import login
import os

# Google Colab에서 토큰 입력 (한 번만 실행)
# https://huggingface.co/settings/tokens 에서 토큰 생성 후 입력
login()

# 또는 직접 토큰 입력 (보안상 권장하지 않음)
# os.environ["HUGGINGFACE_HUB_TOKEN"] = "your_token_here"

In [ ]:
# from huggingface_hub import login
# login("hf_xxxxxxxxxxxxx")

# from transformers import AutoModel
# model = AutoModel.from_pretrained("username/private-model")




---

